In [380]:
import os
import random
import time
import datetime
import torch
import argparse

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [148]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, BertConfig
from transformers import BertForTokenClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [384]:
parser = argparse.ArgumentParser()

In [385]:
def load_data(args):
    temp = pd.read_csv(args, sep="\t",encoding='cp949')
    temp = temp
    document = temp.document.tolist()
    labels = temp.label.tolist()
    return document,labels

In [386]:
path_ = r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\out\all_list.txt"

In [387]:
doc,label = load_data(path_)
doc = [str(i) for i in doc]

# Method 1

In [388]:
files_ = "results_epoch_6_evalAcc_98"

In [389]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=21)
#checkpoint = torch.load(path_to_checkpoint)
checkpoint = torch.load(r"C:\Users\USER\Desktop\All\Project\23_VRP\Classification\model\{}.pth".format(files_))
model.load_state_dict(checkpoint)
device = torch.device("cuda")
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [390]:
# def inference_fn(sentence):
#     inputs = tokenizer(
#         sentence,
#         max_length=20,
#         padding="max_length",
#         truncation=True,
#     )
#     with torch.no_grad():
#         outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
#         probs = outputs.logits[0].softmax(dim=0)
#         top_probs, preds = torch.topk(probs, dim=0, k=1)
#         tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
#         predicted_tags = [ pred.item() for pred in preds]
#         result = []
#         for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
#             if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
#                 token_result = {
#                     "token": token,
#                     "predicted_tag": predicted_tag,
#                     "top_prob": str(round(top_prob[0].item(), 4)),
#                 }
#                 result.append(token_result)
#     return {
#         "sentence": sentence,
#         "result": result,
#     }

In [391]:
#test_1 = doc[3:5]
test_1 = "생선"
#test_1 = doc[3]

test_1

'생선'

In [392]:
def inference_doc(test_1):
    inputs = tokenizer(
        [test_1],
        max_length=20,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=0)
        top_probs, preds = torch.topk(probs, dim=0, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [ pred.item() for pred in preds]

        out_prob = str(round(top_probs[0].item(), 4))
        out_tag = preds.item()
    return [test_1,out_tag,out_prob]

In [395]:
inference_doc(test_1)

['생선', 5, '0.9973']

In [396]:
tests_ = doc[0:20]

In [397]:
out_list = []
tq = tqdm(doc)
for test_ in tq:
    out_list.append(inference_doc(test_))

  0%|          | 0/28148 [00:00<?, ?it/s]

In [398]:
out_df = pd.DataFrame(out_list,columns=["document","label","prob"])
out_df

,document,label,prob
0,nan,2,0.9985
1,완도 명가 다시마(),8,0.9995
2,콜라개,2,0.9964
3,한우사골(국내산 팩),8,0.9994
4,누룽지 100t,8,0.9997
...,...,...,...
28143,포카리스웨트 분말,15,0.9989
28144,artisan biscuits 투바이투 바닐라 비스킷,11,0.9995
28145,양말.스타킹개,15,0.8608
28146,더쫄깃한진공반죽국수,8,0.2622


In [399]:
out_df.to_excel(r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all_cate.xlsx")